In [1]:
import pandas as pd
import os
from yt_dlp import YoutubeDL

In [2]:
# Output directory
output_directory = 'E:/videos'
os.makedirs(output_directory, exist_ok=True)

In [3]:
# Load the Dataset
df = pd.read_excel('MM-Trailer_dataset.xlsx')

In [4]:
# turn the IMDBid and URL column into a list
urls = df['URL'].tolist()
ids = df['IMDBid'].tolist()

In [5]:
# List to keep track of failed downloads
failed_downloads = []

In [6]:
# Download the videos
for id, url in zip(ids, urls):
    # Define the output filename
    output_filename = os.path.join(output_directory, f'{id}.mp4')

    # Check if the file already exists
    if os.path.exists(output_filename):
        print(f"File {output_filename} already exists. Skipping download.")
        continue

    try:
        # Define yt-dlp options with the IMDB ID as the filename
        ydl_opts = {
            'outtmpl': output_filename,
            'format': 'bestvideo+bestaudio/best', 
            'merge_output_format': 'mp4',
            'postprocessors': [
            {  # Use ffmpeg for postprocessing
                'key': 'FFmpegVideoConvertor',
                'preferedformat': 'mp4', # Convert to mp4

            }],

            'noplaylist': True,                   # Do not download playlists
            'concurrent_fragments': 10,            # Download multiple fragments at once
            'retries': 10,                        # Retry download up to 10 times
            'fragment_retries': 10,               # Retry fragments up to 10 times
            'http_chunk_size': 10 * 1024 * 1024,  # Set chunk size to 10 MB
            'sleep_interval_requests': 1,         # Sleep 1 second between requests
            'sleep_interval': 1,                  # Sleep 1 second between downloads
            'socket_timeout': 90,
            'verbose': True,
        }
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
    except Exception as e:
        print(f"Error downloading {url} with IMDB ID {id}: {e}")
        # Add the failed download to the list
        failed_downloads.append((id, url, str(e)))

[debug] Encodings: locale cp1252, fs utf-8, pref cp1252, out UTF-8 (No VT), error UTF-8 (No VT), screen UTF-8 (No VT)
[debug] yt-dlp version stable@2024.05.27 from yt-dlp/yt-dlp [12b248ce6] (pip) API
[debug] params: {'outtmpl': 'E:/videos\\tt0200469.mp4', 'format': 'bestvideo+bestaudio/best', 'merge_output_format': 'mp4', 'postprocessors': [{'key': 'FFmpegVideoConvertor', 'preferedformat': 'mp4'}, {'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3'}], 'noplaylist': True, 'concurrent_fragments': 10, 'retries': 10, 'fragment_retries': 10, 'http_chunk_size': 10485760, 'sleep_interval_requests': 1, 'sleep_interval': 1, 'socket_timeout': 90, 'verbose': True, 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}}
[debug] Python 3.12.

File E:/videos\tt0790799.mp4 already exists. Skipping download.
File E:/videos\tt0478087.mp4 already exists. Skipping download.
File E:/videos\tt1629439.mp4 already exists. Skipping download.
File E:/videos\tt1649444.mp4 already exists. Skipping download.
File E:/videos\tt0147800.mp4 already exists. Skipping download.
File E:/videos\tt0214388.mp4 already exists. Skipping download.
File E:/videos\tt1136683.mp4 already exists. Skipping download.
File E:/videos\tt0211181.mp4 already exists. Skipping download.
File E:/videos\tt0360323.mp4 already exists. Skipping download.
File E:/videos\tt4172430.mp4 already exists. Skipping download.
File E:/videos\tt2059171.mp4 already exists. Skipping download.
File E:/videos\tt0974661.mp4 already exists. Skipping download.
File E:/videos\tt0841044.mp4 already exists. Skipping download.
File E:/videos\tt0086837.mp4 already exists. Skipping download.
File E:/videos\tt1132130.mp4 already exists. Skipping download.
File E:/videos\tt1232829.mp4 already exi

[debug] exe versions: ffmpeg 2024-06-21-git-d45e20c37b-full_build-www.gyan.dev (setts), ffprobe 2024-06-21-git-d45e20c37b-full_build-www.gyan.dev
[debug] Optional libraries: Cryptodome-3.20.0, brotli-1.1.0, certifi-2024.02.02, mutagen-1.47.0, requests-2.32.3, sqlite3-3.45.1, urllib3-2.2.1, websockets-12.0
[debug] Proxy map: {}
[debug] Request Handlers: urllib, requests, websockets
[debug] Loaded 1820 extractors


[youtube] Extracting URL: https://www.youtube.com/watch?v=3NWlsXlHT9Q
[youtube] 3NWlsXlHT9Q: Downloading webpage
[youtube] Sleeping 1 seconds ...
[youtube] 3NWlsXlHT9Q: Downloading ios player API JSON


ERROR: [youtube] 3NWlsXlHT9Q: Video unavailable
  File "c:\Users\Kyle Castro\AppData\Local\Programs\Python\Python312\Lib\site-packages\yt_dlp\extractor\common.py", line 734, in extract
    ie_result = self._real_extract(url)
                ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Kyle Castro\AppData\Local\Programs\Python\Python312\Lib\site-packages\yt_dlp\extractor\youtube.py", line 4248, in _real_extract
    self.raise_no_formats(reason, expected=True)
  File "c:\Users\Kyle Castro\AppData\Local\Programs\Python\Python312\Lib\site-packages\yt_dlp\extractor\common.py", line 1257, in raise_no_formats
    raise ExtractorError(msg, expected=expected, video_id=video_id)

[debug] Encodings: locale cp1252, fs utf-8, pref cp1252, out UTF-8 (No VT), error UTF-8 (No VT), screen UTF-8 (No VT)
[debug] yt-dlp version stable@2024.05.27 from yt-dlp/yt-dlp [12b248ce6] (pip) API
[debug] params: {'outtmpl': 'E:/videos\\tt4503510.mp4', 'format': 'bestvideo+bestaudio/best', 'merge_output_format': 'mp4', '

...

[info] jL3cBIg-eyQ: Downloading 1 format(s): 248+251


[debug] Invoking dashsegments downloader on "https://manifest.googlevideo.com/api/manifest/dash/expire/1719123433/ei/iWl3ZpmqOKC02roP54qtcA/ip/2001%3A4452%3A3a3%3A4500%3Ad12d%3A213f%3A912d%3A256b/id/8cbddc04883e7b24/source/youtube/requiressl/yes/xpc/EgVo2aDSNQ%3D%3D/playback_host/rr5---sn-2aqu-n4xe.googlevideo.com/mh/Mf/mm/31%2C29/mn/sn-2aqu-n4xe%2Csn-2aqu-hoale/ms/au%2Crdu/mv/m/mvi/5/pl/47/hfr/all/as/fmp4_audio_clear%2Cwebm_audio_clear%2Cwebm2_audio_clear%2Cfmp4_sd_hd_clear%2Cwebm2_sd_hd_clear/initcwndbps/1065000/spc/UWF9f63--V8r6xOoiHakDcejbKsZHNI556JT2CZhlXKKSNFNHgeBmw5wjjvM/vprv/1/rqh/2/mt/1719101328/fvip/7/keepalive/yes/itag/0/sparams/expire%2Cei%2Cip%2Cid%2Csource%2Crequiressl%2Cxpc%2Chfr%2Cas%2Cspc%2Cvprv%2Crqh%2Citag/sig/AJfQdSswRQIhAPLOnXTyGDZgForIv-qOkpNFCdBlzk4Rfc_cVnlkQaVoAiAIVmpgkThmPBBL9JLNl2XbHb19vCPwsR6XzRUi5CQByg%3D%3D/lsparams/playback_host%2Cmh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps/lsig/AHlkHjAwRQIhAOFkgD2HiplhV-K1XomywemIeJQMYGh-IzCbVz_YWO5RAiBzCYk0v8shAGiPGs

[download] Sleeping 1.00 seconds ...
[dashsegments] Total fragments: 19
[download] Destination: E:\videos\tt3136168.f248.webm


[debug] File locking is not supported. Proceeding without locking


[download] Got error: HTTPSConnectionPool(host='rr5---sn-2aqu-n4xe.googlevideo.com', port=443): Read timed out.. Retrying (1/10)...
[download]   5.7% of ~   8.79MiB at  304.84KiB/s ETA 00:06 (frag 1/19)  

In [ ]:
# Print the list of failed downloads
if failed_downloads:
    print("\nFailed Downloads:")
    for imdb_id, url, error in failed_downloads:
        print(f"IMDB ID: {imdb_id}, URL: {url}, Error: {error}")
else:
    print("All downloads completed successfully.")